# Introduction

We have done the bulk of the work in getting the data ready to be analyzed. Now it is time to do some exploratory analysis to better understand what information might be in the data. In this notebook we will explore the data, ask some questions about the data and "wrangle" it to make useful columns and variables for further analysis.

# Import & Ingest

As part of this notebook, we are going to being visualizing some of the data. This often makes it easier to spot interesting trends or outliers. The library that we are going to use for visualization is matplotlib. The syntax for building a chart with matplotlib takes some getting used to, but the flexibility that it provides is great once you get used to it. There are many other visualization libraries like plotly, bokeh, and seaborn which are all quite popular as well. Each has its own style and syntax and can be better for certain uses, but in most cases it ultimately comes down to personal (or organizational) preference.

In [ ]:
# Import Libraries

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

Let's start by importing our clean data called clean_data.csv

In [ ]:
# Exercise

filepath = None
df = None

# EDA

Now that we have imported the data, let's explore it a little bit. Let's start with looking at what we have in the airport column. Let's list out all the unique values. Recall that we already converted any lower case characters to upper case, so we shouldn't have to worry about inconsistent formatting.

In [ ]:
unique_airports = df['airport'].unique()
print(unique_airports)

The first thing we notice is that all of the values start with the letter K. Because we know that we are looking at 4 US-Based airlines and because the 3 following letters are all common US airport names (e.g., DFW = Dallas-Fort Worth Airport), we can infer that all of these must be US airports. With a little bit of research (or your own background knowledge), we can learn that [international standards for airport identifiers](https://en.wikipedia.org/wiki/ICAO_airport_code) require a 'K' as the first character for airport codes in the continental US. 

We also see that there are only 34 unique airports represented. This is clearly a subset of even major commercial US airports, so we know we're not getting a full picture of domestic air traffic. All of the largest domestic airports appear to be covered, so we will assume that the trends at these airports are representative of overall trends in *commercial* air traffic. If, however, our analysis were interested in general aviation trends, this would probably not be a sufficient data set.

One additional aside: it is important to track and document these types of assumptions or constraints on the data. Some assumptions may seem trivial now, but if your analysis objectives change later in the process, they can become very important.

Now let's take a look at the destination airports.

In [ ]:
# Exercise 

unique_destination_airports = None
print(unique_destination_airports)

Wow, our list of airports just got a lot longer. This makes sense, however - flights departing from the 34 major airports go to hundreds of airports around the world. We would expect to see the same thing for departure airports (try it!). We also see a many airport with a leading character other than 'K'. That makes sense too - these must be international flights departing from a US airport and arriving at an airport in another country.

We can use this knowledge to create a variable to determine if a flight is domestic (we'll tag these as 'local') or international, which may be interesting to look at later on. Before you peek at the code below, what is the logic you would use to make this determiniation based on departure and destination airport codes? 

In [ ]:
df['type_of_flight'] = 'local'
df.loc[(df['destination_airport'].str[0] != 'K') | (df['departure_airport'].str[0] != 'K'),'type_of_flight'] = 'international'

You'll notice that we first set the new column called type_of_flight to the value of 'local' for all flights and then overwrite it just for the international flights. This avoids us having to filter the flights twice. Be careful, though - this only works if you know that each record fits neatly into one of the two categories.

Next let's look at the call sign column.

In [ ]:
df['call_sign'].head()

The call signs are used to identify the individual flights for passengers, airlines, and air traffic controllers. In general, for commercial flights the first 3 characters of the call sign indicate which airline is operating the flight. So let's look at which airlines are represented here.

In [ ]:
df['call_sign'].str[:3].unique()

One thing you'll notice in this code here is that we used the str operator again. The brackets that follow [:3] indicate that we only want the first 3 letters. Because python is zero indexed and stops before the last index, the [:3] will take the 0th, 1st, and 2nd character. In other words up to the 3rd character in the string. For more information about this, you can look here, https://realpython.com/python-strings/.

To understand this a bit better, let's do a quick exercise. How would you find the last 2 characters in the call_sign column?

In [ ]:
# Exercise
unique = None
unique

Alright, back to the airport data. We see that there are 4 different airlines represented: United, Delta, Southwest and American Airlines. That might be a worrisome result if we thought we had a full picture of traffic at these airports. But in this case, for the purposes of this course, we've intentionally downsampled to just these 4 airlines to reduce the size of the dataset.

Let's make a separate column so that the airline of a flight can be used for later analyses.

In [ ]:
# Exercise

df['airline'] = None

What you've just done is called "feature engineering" - creating new variables useful for analysis or modeling from existing variables in the data set. Feature engineering could include rounding values or converting units of measurement, computing averages over multiple numerical values, adding fields for month/quarter/day of week based on a date or timestamp, creating groupings of data points and adding a group identifier to each record, and on and on and on...

Feature engineering is often an iterative process. We do some up front, but as we see where the analysis takes us, we can come back to create additional features that will help us derive important insights. Good feature engineering can open up entire new directions of analysis.

Now that we have created some columns that might be useful, let's ask ourselves some questions and try to answer them with the data. This allows us to gain a greater intuitive understanding of the data itself and any potential trends.

### What are the busiest airports?

To answer this question, let's look at which airports have the most landings and takeoffs. This is where pandas's *groupby* functionality comes in handy. We will group the observations by airport and then count up the number of data points in each group. Then we will plot this data, sorted from busiest to least busy. 

In [ ]:
df_busy_airports = df.groupby(['airport'])['airport'].count().reset_index(name = 'count').sort_values(['count'], ascending = False)

Before moving on to plotting the graph, we should pause and talk about how we created this dataset to be plotted. 

* The *groupby()* method buckets the data points by airport.
* We then grab just the airport column.
* Next, we apply the *count()* method to each of these groups (airports) to count the number of instances of each. 
* We reset the index. This is a technical step - we need to make the airport names part of the new dataframe of counts rather than the airport names in the index... don't worry if that doesn't make any sense right now. 
* Finally, we sort the records by event counts using the *sort_values()* method in descending order.

The result is a sorted list of airports by the number of times they appear in the airport column in the data set. Now we're ready to create our first plot!

As we said at the top, matplotlib's plotting syntax takes some getting used to. Take some time to understand each line of code below, and try editing different inputs and rerunning to see how it changes the chart that is created.

In [ ]:
# create the basic chart with the first line
plt.bar(x = df_busy_airports['airport'], height = df_busy_airports['count'])

# modify specific elements with subsequent lines of code
plt.title('Busiest Airports')
plt.xlabel('Airport')
plt.ylabel('Count of Events')
plt.xticks(rotation=90);

Since we're comparing values across categories (airports), we used a barplot. This gives a good first view of how the events in the data are spread across airports. We can see a few of the busiest airports that stand out above the others, a lot of airports clustered in the middle, and a quick tailoff in flights volumes for the 7 least busy airports on our list.

Let's see if we can drill down on the 10 busiest airports to take a closer look.

In [ ]:
plt.bar(x = df_busy_airports['airport'].head(10), height = df_busy_airports['count'].head(10))
plt.title('Top 10 Busiest Airports')
plt.xlabel('Airport')
plt.ylabel('Count of Events')
plt.xticks(rotation=45);

This graph makes it easier to evaluate individual airports. We see from the graph that the Atlanta airport is clearly the busiest for these 4 airlines with Dallas-Fort Worth and Denver a distant second and third.

Let's look at the least busiest airports. How would we do that?

In [ ]:
# Exercise
 
df_least_busy_airports = None

In [ ]:
plt.bar(x = df_least_busy_airports['airport'].head(10), height = df_least_busy_airports['count'].head(10))
plt.title('10 Least Busy Airports')
plt.xlabel('Airport')
plt.ylabel('Count of Events')
plt.xticks(rotation=45);

Great! Before we go on, I will explain a bit more about the syntax for plotting bar charts. 

* *plt.* refers to the matplotlib library that we imported at the beginning of the notebook.
* *bar()* is a method from the matplolib library indicating that we would like to generate a bar plot. Other alternatives are *plot()* for scatter or lineplots and *hist()* for histograms. The input parameters to the *bar()* method are:
  * *x*: the data that you want to represent the x-axis
  * *height*: the data that you want for the height of the bars
* *plt.title()*, *plt.xlabel()*, and *plt.ylabel()* are all additional methods from the matplotlib library to give the bar chrat, a title, x-axis label, and y-axis label, respectively.

What happens if we just look at counts for international flights by airport? Fill in the blanks below, including giving the chart a descriptive title.

In [ ]:
index = df['type_of_flight'] == 'international'
df_busy_airports_international = df[index].groupby(['airport'])['airport'].count().reset_index(name = 'count').sort_values(['count'], ascending = False)

plt.bar(x = df_busy_airports_international['airport'], height = df_busy_airports_international['count'])
plt.title('Busiest Airports for International Flights')
plt.xlabel('Airport')
plt.ylabel('Count of Events')
plt.xticks(rotation=90);

What do you notice? KMIA (Miami) was outside of the top 10 for total flight operations but is far and away the busiest airport for international operations. Similarly, JFK jumped from the bottom 10 to number 6. There is a clear dropoff in international flights after the top 8, which are all majore international hubs for one or more US carriers.

One important caveat to remember is that we only have data for the four largest US carriers. Airports like JFK see a lot of operations from international carriers, so our data set may be missing more international operations at JFK, for example, than at other airports. These are the kinds of biases that an analyst has to be aware of when interpreting resuts.

Now that we learned a bit about how to plot. Let's take this one step further and look at airline operation counts at each airport. How would you do this? Hint: You would need to change the columns that you group by and the columns that you sort by.

In [ ]:
# Exercise

df_busy_airports_by_airline = None

Instead of visualizing it, we are simply going to list out the top 10. This is a way to get a quick look . You will notice that is is much harder to gain a understanding of the relative proportions of the airports because there isn't a visual aspect to a table. 

In [ ]:
df_busy_airports_by_airline.head(10)

Once you get more comfortable with graphing and basic data wrangling. You will eventually be able to create more complicated graphs. For example in the graph below, we are able to show the distribution of airports for each airline and see how they differ in one chart. Moreover, it is interactive so you will be able to hover and see the exact numbers for each airline and airport as well as zoom in and out on portions of the graph.

Don't worry about the syntax here - we just want to give you an idea of what is possible.

In [ ]:
fig = px.bar(df_busy_airports_by_airline, x = 'airport', y = 'count', facet_row = 'airline', color = 'airline',
             title = 'Faceted Bar Chart - Busiest Airport by Airline')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''
fig.update_layout(
    annotations = list(fig.layout.annotations) + 
    [go.layout.Annotation(
            x=-.1,
            y=0.5,
            font=dict(
                size=14
            ),
            showarrow=False,
            text="Count of Flights",
            textangle=-90,
            xref="paper",
            yref="paper"
        )
    ]
)        
fig.show()

Moving on, instead of airlines, we can also slice the data by domestic vs. international flights. Let's see if we can find the most common domestic and international flight routes based on the departure and destination airports.

In [ ]:
# Exercise

df_tof = None
plt.bar(x = None, height = None)
plt.title(None)
plt.xlabel(None)
plt.ylabel(None);

As you may have noticed, when you plot a graph with large numeric values, the y-axis doesn't show the entire number but instead converts scientific notation (e.g., 3 x 10e6 instead of 3000000). It does this to save space on the labels, but this can make it harder to interpret, especially if you might share the chart with a non-technical audience. One way to combat this is to scale the input data and note the scaling in the axis title. In the answer, you'll see that we divided the count by 1000000 and then in the label explicitly mentioned that the numbers are in millions.

Now let's see if you can find the most traveled routes both within and outside the continental USA. Let's just list out the top 6 most common routes in each category.

In [ ]:
# Exercise
# Domestic Flights

common_local_flights = None
print(common_local_flights)

# Interational Flights

common_international_flights = None
print(common_international_flights)

The top domestic route is KDEN (Denver) to KPHX (Phoenix). The top "international" route is PHNL (Honolulu) to KLAX (Los Angeles) - we've stumbled across a problem with our designation of international vs. domestic flights! Depending on the goal of the analysis, that may or may not be OK. But this is another assumption that we should note in case it becomes important later, and we should clearly communicate this with our analysis results if it has any impact on the interpretation.

There seems to be a nice pairing of routes. For example, KDEN-KPHX and its reverse route KPHX-KDEN have similarly high flight counts. This makes sense - if there are a lot of people traveling in one direction on a route, there are probably a similar number of people that need to travel in the other direction on that route. The one outlier seems to be KATL (Atlanta) to KMCO (Orlando). Can you find the flight counts for the reverse route? 

In [ ]:
# Exercise
index = None
common_domestic_flights[index]

The reverse flight is only 286 flights (about 5%) behind the KATL-KMCO route - coming in at #8 on the list, as it turns out.

Nice Job! After this notebook, you should have started to gain a familiarity with this dataset, learning how to plot a bar chart and how to group and count observations. In the next notebook, we will look at more ways to visualize data as well as deal with time. 